## Logistic Regression - 8K Forms & Numerical Features

In [ ]:
import time
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import nltk
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
import sklearn.preprocessing as preproc
from sklearn.feature_extraction import text
import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/with_all_ft_clean.pkl")
df.head()

In [ ]:
def label1(row):
    if row["pct_change1"] > row["mkt_excess1"]:
        return "pos"
    else:
        return "neg"

def label2(row):
    if row["pct_change2"] > row["mkt_excess2"]:
        return "pos"
    else:
        return "neg"

def label3(row):
    if row["pct_change2"] > row["mkt_excess2"] + 0.005:
        return "positive"
    elif row["pct_change2"] < row["mkt_excess2"] - 0.005:
        return "negative"
    else:
        return "neutral"

In [ ]:
df["label1"] = df.apply(label1, axis = 1)
df["label2"] = df.apply(label2, axis = 1)
df["label3"] = df.apply(label3, axis = 1)

In [ ]:
# get column names of categorical features
item_list = [i for i in df.columns if not i.find("item")]
ind_list = [i for i in df.columns if not i.find("ind")]
shrcd_list = [i for i in df.columns if not i.find("SHRCD")]
exc_list = [i for i in df.columns if not i.find("EXCHCD")]

In [ ]:
# combine all feature column names
cols = ['VIX', "Mkt_Cap"]
cols.extend(item_list)
cols.extend(ind_list)
cols.extend(shrcd_list)
cols.extend(exc_list)

#### To get 1-day binary, 2-day binary, and multinomial labels, column name can be changed as "label1", "label2", "label3":

In [ ]:
# Separate into features, documents and label
cols = ['VIX', "Mkt_Cap"]
cols.extend(item_list)
cols.extend(ind_list)
cols.extend(shrcd_list)
cols.extend(exc_list)

X = df[cols]

docs = df['texts']

#change the label for different outcomes
y = df['label2']

#### Process texts

In [ ]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    # Tokenize each word
    text =  nltk.WordPunctTokenizer().tokenize(text)
        
    return text

In [ ]:
# apply clean function to documents
docs = df.texts.apply(clean_text)
docs.tail()

21697    [item, 2, 02, results, operations, financial, ...
21698    [item, 5, 07, submission, matters, vote, secur...
21699    [item, 5, 02, departure, directors, certain, o...
21700    [item, 2, 02, results, operations, financial, ...
21701    [item, 5, 07, submission, matters, vote, secur...
Name: texts, dtype: object

In [ ]:
# lemmatize documents
def lemmatized_words(text):
    lemm = nltk.stem.WordNetLemmatizer()
    return list(map(lambda word: list(map(lemm.lemmatize, word)), docs))
    
lemmatized = lemmatized_words(docs)

In [ ]:
df.label2.value_counts()

pos    12742
neg     8960
Name: label2, dtype: int64

In [ ]:
# Split into train and test data
X_train, X_test, y_train, y_test, docs_train, docs_test = train_test_split(X, y, lemmatized,
                                                    stratify=y, 
                                                    test_size=0.2,
                                                    random_state = 20)

In [ ]:
# standardize volatility index and market cap by mean and standard deviation (standardize train and test sets separately)
x_scaler = StandardScaler()
X_train["VIX"] = x_scaler.fit_transform(np.array(X_train["VIX"]).reshape(-1,1))
X_test["VIX"] = x_scaler.transform(np.array(X_test["VIX"]).reshape(-1,1))
X_train["Mkt_Cap"] = x_scaler.fit_transform(np.array(X_train["Mkt_Cap"]).reshape(-1,1))
X_test["Mkt_Cap"] = x_scaler.transform(np.array(X_test["Mkt_Cap"]).reshape(-1,1))

In [ ]:
# define BoW transformer, set max_features to 4000 to balance the dimensions with numerical features
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, max_features=4000, ngram_range=[1,3], lowercase=False)

In [ ]:
#transform documents
X_tr_bow = bow_transform.fit_transform(docs_train)
X_te_bow = bow_transform.transform(docs_test)

In [ ]:
X_tr_bow.shape

(17361, 4000)

In [ ]:
#concatenate BoW features with numerical features (convert Bow features to dense array first)
X_tr_bow_ft = np.concatenate((X_tr_bow.toarray(), np.array(X_train)), axis = 1)
X_te_bow_ft = np.concatenate((X_te_bow.toarray(), np.array(X_test)), axis = 1)

In [ ]:
# define Tf-idf transformer, set max_features to 4000 to balance the dimensions with numerical features
tfidf_transform = TfidfVectorizer(tokenizer=lambda doc: doc, max_features=4000, ngram_range=[1,3], lowercase=False)

In [ ]:
#transform documents
X_tr_tfidf = tfidf_transform.fit_transform(docs_train)
X_te_tfidf = tfidf_transform.transform(docs_test)

In [ ]:
#concatenate Tf-idf features with numerical features (convert Tf-idf features to dense array first)
X_tr_tfidf_ft = np.concatenate((X_tr_tfidf.toarray(), np.array(X_train)), axis = 1)
X_te_tfidf_ft = np.concatenate((X_te_tfidf.toarray(), np.array(X_test)), axis = 1)

In [ ]:
# define logistic regression classifier
def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
    model = LogisticRegression(C=_C).fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', description, 'features', score)
    return model

In [ ]:
# fit the model to (BoW + numerical) and (Tf-idf + numerical) features
model_bow = simple_logistic_classify(X_tr_bow_ft, y_train, X_te_bow_ft, y_test, 'bow')
model_tfidf = simple_logistic_classify(X_tr_tfidf_ft, y_train, X_te_tfidf_ft, y_test, 'tf-idf')

Test Score with bow features 0.7242570836212854
Test Score with tf-idf features 0.7357751670122091


In [ ]:
# Hyperparameter tuning

# grid search parameters
param_grid_ = [{'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2], 'penalty':["l1", "l2", "elasticnet"]}]

#create grid search with 5-fold cross validation
bow_search = sklearn.model_selection.GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)
tfidf_search = sklearn.model_selection.GridSearchCV(LogisticRegression(), cv=5,
                                   param_grid=param_grid_)

In [ ]:
bow_search.fit(X_tr_bow_ft, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0],
                          'penalty': ['l1', 'l2', 'elasticnet']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
bow_search.best_estimator_

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
bow_search.best_score_

0.7233454821327395

In [ ]:
bow_pred = bow_search.predict(X_te_bow_ft)

In [ ]:
print(classification_report(y_test, bow_pred))

              precision    recall  f1-score   support

         neg       0.67      0.65      0.66      1792
         pos       0.76      0.78      0.77      2549

    accuracy                           0.73      4341
   macro avg       0.72      0.71      0.72      4341
weighted avg       0.72      0.73      0.72      4341



In [ ]:
tfidf_search.fit(X_tr_tfidf_ft, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0],
                          'penalty': ['l1', 'l2', 'elasticnet']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
tfidf_search.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
tfidf_search.best_score_

0.7399341330952005

In [ ]:
tfidf_pred = tfidf_search.predict(X_te_tfidf_ft)

In [ ]:
print(classification_report(y_test, tfidf_pred))

              precision    recall  f1-score   support

         neg       0.72      0.58      0.64      1792
         pos       0.74      0.85      0.79      2549

    accuracy                           0.74      4341
   macro avg       0.73      0.71      0.72      4341
weighted avg       0.73      0.74      0.73      4341



### Multi-Class Logistic Regression

In [ ]:
y = df['label3']

We have to repeat the process, because classes (split&stratification) changed.

In [ ]:
# Split into train and test data
X_train, X_test, y_train, y_test, docs_train, docs_test = train_test_split(X, y, lemmatized,
                                                    stratify=y, 
                                                    test_size=0.2,
                                                    random_state = 20)

In [ ]:
X_train["VIX"] = x_scaler.fit_transform(np.array(X_train["VIX"]).reshape(-1,1))
X_test["VIX"] = x_scaler.transform(np.array(X_test["VIX"]).reshape(-1,1))
X_train["Mkt_Cap"] = x_scaler.fit_transform(np.array(X_train["Mkt_Cap"]).reshape(-1,1))
X_test["Mkt_Cap"] = x_scaler.transform(np.array(X_test["Mkt_Cap"]).reshape(-1,1))

In [ ]:
#transform documents
X_tr_tfidf = tfidf_transform.fit_transform(docs_train)
X_te_tfidf = tfidf_transform.transform(docs_test)
X_tr_bow = bow_transform.fit_transform(docs_train)
X_te_bow = bow_transform.transform(docs_test)

In [ ]:
#concatenate text vectors with numerical features (convert sparse matrix to dense matrix first)
X_tr_tfidf_ft = np.concatenate((X_tr_tfidf.toarray(), np.array(X_train)), axis = 1)
X_te_tfidf_ft = np.concatenate((X_te_tfidf.toarray(), np.array(X_test)), axis = 1)
X_tr_bow_ft = np.concatenate((X_tr_bow.toarray(), np.array(X_train)), axis = 1)
X_te_bow_ft = np.concatenate((X_te_bow.toarray(), np.array(X_test)), axis = 1)

In [ ]:
# define logistic regression classifier
def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
    model = LogisticRegression(C=_C, multi_class="multinomial").fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', description, 'features', score)
    return model

In [ ]:
# fit the model to (BoW + numerical) and (Tf-idf + numerical) features
model_bow = simple_logistic_classify(X_tr_bow_ft, y_train, X_te_bow_ft, y_test, 'bow')
model_tfidf = simple_logistic_classify(X_tr_tfidf_ft, y_train, X_te_tfidf_ft, y_test, 'tf-idf')

Test Score with bow features 0.6385625431928127
Test Score with tf-idf features 0.65860400829302


In [ ]:
# Hyperparameter tuning

# grid search parameters
param_grid_ = [{'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2], 'penalty':["l1", "l2", "elasticnet"]}]

#create grid search with 5-fold cross validation
bow_search = sklearn.model_selection.GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)
tfidf_search = sklearn.model_selection.GridSearchCV(LogisticRegression(), cv=5,
                                   param_grid=param_grid_)

In [ ]:
bow_search.fit(X_tr_bow_ft, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0],
                          'penalty': ['l1', 'l2', 'elasticnet']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
bow_search.best_estimator_

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
bow_search.best_score_

0.6307811345189552

In [ ]:
bow_pred = bow_search.predict(X_te_bow_ft)

In [ ]:
print(classification_report(y_test, bow_pred))

              precision    recall  f1-score   support

    negative       0.51      0.43      0.47       690
     neutral       0.69      0.78      0.73      2431
    positive       0.57      0.47      0.52      1220

    accuracy                           0.64      4341
   macro avg       0.59      0.56      0.57      4341
weighted avg       0.63      0.64      0.63      4341



In [ ]:
tfidf_search.fit(X_tr_tfidf_ft, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0],
                          'penalty': ['l1', 'l2', 'elasticnet']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
tfidf_search.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
tfidf_search.best_score_

0.6611945209987248

In [ ]:
tfidf_pred = tfidf_search.predict(X_te_tfidf_ft)

In [ ]:
print(classification_report(y_test, tfidf_pred))

              precision    recall  f1-score   support

    negative       0.57      0.37      0.45       690
     neutral       0.68      0.85      0.75      2431
    positive       0.62      0.45      0.52      1220

    accuracy                           0.66      4341
   macro avg       0.62      0.55      0.57      4341
weighted avg       0.65      0.66      0.64      4341

